In [4]:
import subprocess
import sys
import requests
import time
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from src.llama import llama
from src.postgres_db import VectorDB

In [ ]:

def clone_or_update_github_docs(target_dir, repo_url):
    if os.path.exists(target_dir):
        print(f"Repository already cloned in '{target_dir}'. Pulling latest changes...")
        try:
            result = subprocess.run(
                ["git", "-C", target_dir, "pull"],
                check=True, capture_output=True, text=True
            )

            return result.stdout
        except subprocess.CalledProcessError as e:
            print(f"Error pulling latest changes: {e}")
            print(f"Error details: {e.stderr}")
            return False
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return False
    else:
        print("Cloning GitHub docs repository...")
        try:
            result = subprocess.run(
                [
                    "git",
                    "clone",
                    "--depth=1",
                    "--single-branch",
                    "--branch", "main",
                    repo_url,
                    target_dir,
                ],
                check=True, capture_output=True, text=True
            )
            print(f"Repository cloned successfully to '{target_dir}' directory!")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error cloning repository: {e}")
            print(f"Error details: {e.stderr}")
            return False
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return False


def copy_md_files(src_dir, dest_dir):

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.md') and file not in ['index.md', 'README.md']:
                relative_path = os.path.relpath(root, src_dir)
                dest_subdir = os.path.join(dest_dir, relative_path)
                full_path = relative_path + "/" + file
                                    
                if not os.path.exists(dest_subdir):
                    os.makedirs(dest_subdir)
                
                if os.path.exists(dest_subdir + "/" + file):
                    print(f"File already exists: {file} in {dest_subdir}, skipping fetch.")
                    continue
                
                api = "https://docs.github.com/api/article/body?pathname=/en/" + full_path.replace('.md', '')
                
                result = requests.get(api)
                time.sleep(2)
                if result.status_code == 429:
                    print("Rate limit exceeded. Please try again later.")
                    sys.exit(1)
                if result.status_code == 200:
                    n = n + 1
                    with open(os.path.join(dest_subdir, file), 'w', encoding='utf-8') as f:
                        f.write(result.text)
                    print(f"Fetched and wrote: {full_path}")

def git_helper():
    repo_url = "https://github.com/github/docs.git"
    target_dir = "gh_docs"
    result = clone_or_update_github_docs(target_dir, repo_url)
    copy_md_files(f'{target_dir}/content', 'data')
    
if __name__ == "__main__":
    git_helper()

In [6]:
class document_loader: 
    
    def load_documents(self, DATA_PATH="data"):
        documents = []
        
        for root, dirs, files in os.walk(DATA_PATH):
            for file in files:
                if file.endswith(".md"):
                    try:
                        file_path = os.path.join(root, file)
                        loader = TextLoader(file_path, encoding='utf-8')
                        documents.extend(loader.load())
                    except Exception as e:
                        print(f"Error loading {file}: {str(e)}")

        return documents

    def split_documents(self, documents):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=5000,
            chunk_overlap=100,
            length_function=len,
            is_separator_regex=False,
        )
        return text_splitter.split_documents(documents)
    
doc_loader = document_loader()
documents = doc_loader.load_documents()
sliced_documents = doc_loader.split_documents(documents) 



In [9]:
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://ollama:11434")
EMBEDDING_MODEL = "granite-embedding:30m" # Find available models here https://ollama.com/library
POSTGRES_HOST = os.getenv("POSTGRES_HOST", "vector-postgres")

llm = llama(OLLAMA_HOST, embedding_model=EMBEDDING_MODEL)
vectordb = VectorDB(host=POSTGRES_HOST)
dim=384
vectordb.delete_db()
vectordb.init_db(dim=dim)

n=0
start_time = time.time()
for doc in sliced_documents:
    embedding = llm.create_embedding(f"{doc}")

    if embedding:
        vectordb.insert_document(f"{doc}", embedding)
        if n % 100 == 0 and n != 0:
            print(f"✅ Processed {n}/{len(sliced_documents)} docs in {round(time.time() - start_time)} seconds")
        n += 1
    else:
        print("Failed to create embedding.")

Model 'granite-embedding:30m' is already available in Ollama.
✅ Database deleted successfully!
✅ Database initialized successfully!
✅ Processed 10/6416 docs in 8 seconds
✅ Processed 20/6416 docs in 16 seconds
✅ Processed 30/6416 docs in 21 seconds
✅ Processed 40/6416 docs in 27 seconds
✅ Processed 50/6416 docs in 32 seconds
✅ Processed 60/6416 docs in 37 seconds
✅ Processed 70/6416 docs in 43 seconds
✅ Processed 80/6416 docs in 48 seconds
✅ Processed 90/6416 docs in 53 seconds
✅ Processed 100/6416 docs in 59 seconds
✅ Processed 110/6416 docs in 65 seconds
✅ Processed 120/6416 docs in 71 seconds
✅ Processed 130/6416 docs in 77 seconds
✅ Processed 140/6416 docs in 82 seconds
✅ Processed 150/6416 docs in 88 seconds
✅ Processed 160/6416 docs in 94 seconds
✅ Processed 170/6416 docs in 99 seconds
✅ Processed 180/6416 docs in 105 seconds
✅ Processed 190/6416 docs in 111 seconds
✅ Processed 200/6416 docs in 116 seconds
✅ Processed 210/6416 docs in 122 seconds
✅ Processed 220/6416 docs in 127 s

In [10]:
vectordb = VectorDB(host=POSTGRES_HOST)
llm = llama(OLLAMA_HOST, embedding_model=EMBEDDING_MODEL)

prompt = "How much do GitHub actions cost?"
similar_docs = vectordb.search_similar(prompt, top_k=3)

Model 'granite-embedding:30m' is already available in Ollama.
❌ Error searching documents: malformed vector literal: "How much do GitHub actions cost?"
LINE 2:                 SELECT id, content, embedding <-> 'How much ...
                                                          ^
DETAIL:  Vector contents must start with "[".

